In [ ]:
import time
from functools import wraps

def timeit(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = fn(*args, **kwargs)
        end = time.time()
    
        print("Elapsed Time: ", end - start)
        return result
    
    return wrapper

In [ ]:
# import redis

In [ ]:
# import redis
# r=redis.Redis(host='localhost',port=6379)

In [ ]:
import sqlite3

def setup_classifier_db(conn):
    conn.execute("""
        CREATE TABLE IF NOT EXISTS processed_files
        (
            file_path      TEXT   NOT NULL,
            k_mer_len      INT    NOT NULL,
            k_mer_count   INT    NOT NULL
        );
    """)
    
    conn.execute("""
        CREATE TABLE IF NOT EXISTS k_mers
        (
            k_mer        INT    NOT NULL,
            k_mer_len    INT    NOT NULL,
            category     TEXT   NOT NULL
        );
    """)
    
    # conn.execute("""
    #     CREATE INDEX IF NOT EXISTS k_mer_of_len
    #     ON k_mers(k_mer, k_mer_len);
    # """)
    
    conn.commit()

def get_classifier_db():
    conn = sqlite3.connect("disease_classifier.db")
    setup_classifier_db(conn)
    
    return conn

def create_classifier_index():
    conn = sqlite3.connect("disease_classifier.db")
    
    conn.execute("""
        CREATE INDEX IF NOT EXISTS k_mer_of_len
        ON k_mers(k_mer, k_mer_len);
    """)
    
    conn.commit()
    conn.close()
    

In [ ]:
def encode_k_mer(k_mer):
    k_mer = k_mer.lower()
    k = len(k_mer)
    
    code = 0
    
    val = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    
    for ch in k_mer:
        code = code * 4 + val[ch]
    return code

def decode_k_mer(code, k):
    k_mer_chars = []
    
    valToChar = {0: 'a', 1: 'c', 2: 'g', 3: 't'}
    
    for i in range(k):
        char = valToChar[code % 4]
        code //= 4
        
        k_mer_chars.append(char)
    
    return "".join(k_mer_chars[::-1])

In [1]:
def encode_k_mer(k_mer):
    k_mer = k_mer.lower()
    k = len(k_mer)
    
    code = 0
    
    val = {'a': 0, 'c': 1, 'g': 2, 't': 3}
    
    for ch in k_mer:
        code = code * 4 + val[ch]
    return code

def decode_k_mer(code, k):
    k_mer_chars = []
    
    valToChar = {0: 'a', 1: 'c', 2: 'g', 3: 't'}
    
    for i in range(k):
        char = valToChar[code % 4]
        code //= 4
        
        k_mer_chars.append(char)
    
    return "".join(k_mer_chars[::-1])

In [4]:
import time 

In [8]:
# start_time = time.time()
# print(decode_k_mer(encode_k_mer("TCGCTA"),6))
# print(encode_k_mer("TCGCTA"))
print(decode_k_mer(3484,13))
# print("--- %s seconds ---" % (time.time() - start_time))

# start_time = time.time()
# print(decode_k_mer(encode_k_mer("AAAT"), 4))
# print("--- %s seconds ---" % (time.time() - start_time))

# # print(decode_k_mer(encode_k_mer("AACA"), 4))


# start_time = time.time()
# print(decode_k_mer(encode_k_mer("AACAA"), 5))
# print("--- %s seconds ---" % (time.time() - start_time))
# print(decode_k_mer(encode_k_mer("TgCAA"), 5))


aaaaaaatcgcta


In [ ]:
def load_file_lines(file_path):
    file=open(file_path,"r")
    lines=file.read().splitlines()
    return lines


In [ ]:
def generate_k_mer(line,k):
    for i in range(len(line)-k+1):
        seq=line[i:i+k]
        yield seq

In [ ]:
def extract_k_mers_from_line(line, label, k):
    # k_mer_count = 0
    k_mers = set()
    for k_mer in generate_k_mer(line,k):
        # k_mer_count += save_k_mer(k_mer,label)
        k_mers.add(k_mer)
    
    return k_mers
    # return k_mer_count
    # return (len(line) - k + 1, k_mer_count)

In [ ]:
from multiprocessing import Pool, cpu_count
from itertools import cycle

@timeit
def extract_k_mers(file_path,label,k):
    lines=load_file_lines(file_path)
    
    args = list(zip(lines, cycle([label]), cycle([k])))
    
    pool_size = cpu_count()
    with Pool(pool_size) as p:
        k_mer_sets = p.starmap(extract_k_mers_from_line, args)
        return set.union(*k_mer_sets)
        # k_mer_counts = p.starmap(extract_k_mers_from_line, args)
        # return sum(k_mer_counts) 

In [ ]:
# def save_k_mer(c, k_mer,category):
#     c.execute("""INSERT OR IGNORE INTO k_mers
#         VALUES (?, ?, ?)""", (encode_k_mer(k_mer), len(k_mer), category))
    
#     # key = f"{category}_{k_mer}"
#     # r.set(key, 1)
    
#     return 1

In [ ]:
@timeit
def save_k_mers(k_mers, category):
    with get_classifier_db() as conn:
        c = conn.cursor()
        k_mers = list(set(k_mers))
        
        chunk_size = 100000
        for i in range(0, len(k_mers), chunk_size):
            k_mer_chunk = k_mers[i : i + chunk_size]
            records_block = map(lambda k_mer: (encode_k_mer(k_mer), len(k_mer), category), k_mer_chunk)

            c.executemany("INSERT OR IGNORE INTO k_mers VALUES (?, ?, ?)", records_block)
            
            conn.commit()
            
            print(f"\rStoring k_mers in db... {100*i/len(k_mers):.2f}%", end="")
        
        print("\r                                                       ")


In [ ]:
import os

def is_file_processed(file, k):
    conn = get_classifier_db()
    c = conn.cursor()
    
    c.execute("""SELECT 1 FROM processed_files
        WHERE file_path = ? AND k_mer_len = ?""", (file, k))
    
    rs = c.fetchone()
    return rs is not None
    
    # file_name = os.path.basename(file)
    # status_key = f"status_{k}_{file_name}"
    
    
    # return r.get(status_key) is not None

In [ ]:
def mark_file_processed(file, k, k_mer_count):
    
    with get_classifier_db() as conn:
        c = conn.cursor()

        c.execute("""INSERT INTO processed_files
            VALUES (?, ?, ?)""", (file, k, k_mer_count))

        conn.commit()

        # file_name = os.path.basename(file)
        # status_key = f"status_{k}_{file_name}"

        # r.set(status_key, k_mer_count)

In [ ]:
def processed_k_mer_count(file, k):
    conn = get_classifier_db()
    c = conn.cursor()
    
    c.execute("""SELECT k_mek_mer_countr_count FROM processed_files
        WHERE file_path = ? AND k_mer_len = ?""", (file, k))
    
    rs = c.fetchone()
    return None if rs is None else rs[0]

In [ ]:
import os

def process_file(file, category, k):
    file_name = os.path.basename(file)
    print()
    print(f"k = {k}")
    print(category + " - " + file_name)
    
    if is_file_processed(file, k):
        print("Already  processed, k_mer count:", processed_k_mer_count(file, k))
    else:
        print("Extracting k_mers")
        # k_mer_count = extract_k_mers(file, category, k)
        k_mers = extract_k_mers(file, category, k)
        
        k_mer_count = len(k_mers)
        print("k_mer count:", k_mer_count)
        
        print("Storing k_mers in db...", end = "")
        save_k_mers(k_mers, category)
        
        
        mark_file_processed(file, k, k_mer_count)

In [ ]:
import os

def get_files_and_categories(rootDir):
    root, categories, _ = next(os.walk(rootDir))
    
    files = []
    file_category = dict()
    
    for category in categories:
        directory = os.path.join(root, category)
        
        for file in glob.glob(f"{directory}/*.txt"):
            files.append(file)
            file_category[file] = category
    
    return categories, files, file_category
    

In [ ]:
import os
import glob

# txtfiles = []
# for file in glob.glob("*.txt"):
#     txtfiles.append(file)
def process_all_data_files(k):
    categories, files, file_category = get_files_and_categories("./clean_data")
    
    for file in files:
        category = file_category[file]
        process_file(file, category, k)


In [ ]:
%set_env TMPDIR=/media/bioinfo/Data_1/Preonath_Thesis/MS_thesis/db_tmp
%set_env SQLITE_TMPDIR=/media/bioinfo/Data_1/Preonath_Thesis/MS_thesis/db_tmp

In [ ]:
!echo $TMPDIR
!echo $SQLITE_TMPDIR

In [ ]:
for k in range(9, 30):
    process_all_data_files(k)

create_classifier_index()

In [ ]:
import collections
from collections import defaultdict

def is_kmer_in_category(k_mer, category):
    conn = get_classifier_db()
    c = conn.cursor()
    
    k = len(k_mer)

    c.execute("""SELECT 1 FROM k_mers
        WHERE k_mer = ? AND k_mer_len = ? AND category = ?""", (encode_k_mer(k_mer), k, category))
    
    rs = c.fetchone()
    return rs is not None

def test_model(k):
    categories, _, _ = get_files_and_categories("./clean_data")
    _, files, file_category = get_files_and_categories("./test_data")
    
    
    for file in files:
        print()
        print(file)
        
        actual_category = file_category[file]
        print(actual_category)
        k_mers = extract_k_mers(file, actual_category, k)
        
        category_count = defaultdict(int)
        
        for k_mer in k_mers:
            for category in categories:
                if is_kmer_in_category(k_mer, category):
                    category_count[category] += 1
            
        for category in categories:
            print(category, category_count[category])
        
    
test_model(21)

### k = 9

### 113240320 k_mer keys

./test_data/SRS/final.contigs_SRS011271.fa.txt\
SRS\
Elapsed Time:  54.02953052520752\
[('SRS', 262143), ('CSM', 261977), ('HMP2', 262143), ('MSM', 262139)]

./test_data/CSM/final.contigs_CSM5FZ44_P.fa.txt\
CSM\
Elapsed Time:  5.754974842071533\
[('SRS', 261774), ('CSM', 261654), ('HMP2', 261774), ('MSM', 261771)]

./test_data/HMP2/final.contigs_HMP2_J00829_M_ST_T0_B0_0120_ZN9YTFN-1014_C8GAAANXX_L002.fa.txt\
HMP2\
Elapsed Time:  63.694913387298584\
[('SRS', 262144), ('CSM', 261978), ('HMP2', 262144), ('MSM', 262140)]

In [ ]:
# @timeit
# def deleteAll():
#     keys = r.keys("*")
#     for key in keys:
#         r.delete(key)
#     print(len(keys))
    
# deleteAll()

In [ ]:
# !pip install redis

In [ ]:
# import redis
# import time 
# r=redis.Redis(host='localhost',port=6379)
# # r.set("France","Paris")
# # r.set("Germany","Berlin")
# capital_france=r.get("France")
# capital_germany=r.get("Germany")
# print(capital_france)
# print(capital_germany)